In [0]:
%run /Workspace/Users/ebadurrehman2012@gmail.com/Databricks-Certified-Data-Engineer-Associate/Includes/Copy-Datasets

In [0]:
(spark.readStream.table("books")
.createOrReplaceTempView("books_streaming_tmp_vu")

)

In [0]:
%sql
select * from books_streaming_tmp_vu

In [0]:
%sql
select author,count(book_id) as total_books from books_streaming_tmp_vu group by author

In [0]:
%sql
create or replace temp view author_count_tmp_vw as (
  select author,count(book_id) as total_books from books_streaming_tmp_vu group by author
)

In [0]:
|%sql
select * from author_count_tmp_vw

In [0]:
(
    spark.table("author_count_tmp_vw")
    .writeStream
    .trigger(processingTime='4 seconds')
    .outputMode("complete")
    .option("checkpointLocation", "dbfs:/mnt/demo/author_counts_checkpoint")
    .table("author_count")
)

In [0]:
%sql
select * from author_count

In [0]:
dbutils.fs.mkdirs('dbfs:/mnt/demo')

In [0]:
%sql
describe extended author_count_tmp_vw

In [0]:
%fs ls dbfs:/mnt/demo-datasets/bookstore/orders-streaming/

In [0]:
%sql
INSERT INTO books
values
("B19", "Introduction to Modeling and Simulation", "Mark W. Spong", "Computer Science", 25),
("B20", "Robot Modeling and Control", "Mark W. Spong", "Computer Science", 30),
("B21", "Turing's Vision: The Birth of Computer Science", "Chris Bernhardt", "Computer Science", 35)

In [0]:
%sql
INSERT INTO books
values ("316", "Hands-On Deep Learning Algorithms with Python", "Sudharsan Ravichandiran", "Computer Science", 25), ("B17", "Neural Network Methods in Natural Language Processing", "Yoav Goldberg", "Computer Science", 30), ("B18", "Understanding digital signal processing", "Richard Lyons", "Computer Science", 35)

In [0]:
(spark.table("author_count_tmp_vw")
.writeStream
.trigger(availableNow=True)
.outputMode("complete")
.option("checkpointLocation", "dbfs:/mnt/demo/author_counts_checkpoint")
.table("author_counts")
.awaitTermination()
)

In [0]:
%sql
select * from author_count

In [0]:
%sql 
INSERT INTO books
values ("B16", "Hands-On Deep Learning Algorithms with Python", "Sudharsan Ravichandiran", "Computer Science", 25),
("B17", "Neural Network Methods in Natural Language Processing", "Yoav Goldberg", "Computer Science", 38), ("818", "Understanding digital signal processing", "Richard Lyons", "Computer Science", 35)